## Extract, Transform, Load (ETL) Project


Data was sourced from openFDA - the US Food and Drug Administration (USFDA) API. We selected animal and veterinary API endpoints. The available API was the Adverse Event Report. During our EDA project, we selected data on a small set of attributes of cats, recorded from 1987-2021. However, the same code no longer would run so we switched to selecting data on dogs. 

We obtained an API key, defined a base url, an made API calls. The output was converted to json format. The key was not used for most of the data retrieval. Data extraction was facilitated by a loop.  The original 750 records were stored in a dataframe and also saved as a .csv file.

Our dataframe comprises reports of incidents where drug exposure resulted in adverse reactions in dogs. After inspecting the variables, we trimmed the dataframe to a set of 415 records. 

### Retrieving data and creating dataframe

In [1]:
#Setting dependencies

import numpy as np
import pandas as pd

import json
import requests
import time

import matplotlib.pyplot as plt
from pandas.plotting import table
from pprint import pprint
import seaborn as sns

# Import API key

from api_keys import api_key

In [3]:
# This url returns only the first match, so we used a loop to skip entries and pick up another one.otherwise the 
# same one is returned every time.
# Returning 750 results runs for a bit of time but will provide us with more data to look at/scrub
#creating empty dictionary to store extracted data

cats_data = {'species':[],
                'gender':[],
                'age':[],
                'weight':[],
                'breed':[],
                'drug':[],
                'outcome':[],
                'date_in':[],
                'react_term_code':[],
                'react_term_name':[],
                'first_exp_da':[],
                'last_exp_da':[],
                'admin_by':[],
                'route':[],
                'dosage_form':[]}
#base_url = "https://api.fda.gov/animalandveterinary/event.json?search=animal.species=Cat'"
base_url = "https://api.fda.gov/animalandveterinary/event.json?search=animal.breed.breed_component<>'Unknown'"
# NB: It turns out that the API key was unnecessary - the + api_key part of the call was not copied, and the call was successful. These data were extracted without using the key.

counter = 1
for i in range(750):
    req = requests.get(base_url)
    data = req.json()

    # validate the data, if age and weight don't exist, replace with 0. If date fields don't exist, replace with nan.
    # For all other fields that don't exist, replace with 'Unknown'. 
    try:
        cats_data['age'].append(data['results'][0]['animal']['age']['min'])
    except:
        cats_data['age'].append('0')
    
    try:
        cats_data['weight'].append(data['results'][0]['animal']['weight']['min'])
    except:
        cats_data['weight'].append('0')

    try:
        cats_data['outcome'].append(data['results'][0]['outcome'][0]['medical_status'])
    except:
        cats_data['outcome'].append('Unknown')
    
    try:
        cats_data['date_in'].append(data['results'][0]['original_receive_date'])
    except:
        cats_data['date_in'].append(np.nan)

    try:
        cats_data['species'].append(data['results'][0]['animal']['species'])
    except:
        cats_data['species'].append('Unknown')
        
    try:
        cats_data['gender'].append(data['results'][0]['animal']['gender'])
    except:
        cats_data['gender'].append('Unknown')
    
    try:
        cats_data['breed'].append(data['results'][0]['animal']['breed']['breed_component'])
    except:
        cats_data['breed'].append('Unknown')
    
    try:
        cats_data['drug'].append(data['results'][0]['drug'][0]['active_ingredients'][0]['name'])
    except:
        cats_data['drug'].append('Unknown')
        
    try:    
        cats_data['react_term_code'].append(data['results'][0]['reaction'][0]['veddra_term_code'])
    except:
        cats_data['react_term_code'].append('Unknown')
    
    try:
        cats_data['react_term_name'].append(data['results'][0]['reaction'][0]['veddra_term_name'])
    except:
        cats_data['react_term_name'].append('Unknown')
    
    try:
        cats_data['first_exp_da'].append(data['results'][0]['drug'][0]['first_exposure_date'])
    except:
        cats_data['first_exp_da'].append(np.nan)
        
    try: 
        cats_data['last_exp_da'].append(data['results'][0]['drug'][0]['last_exposure_date'])
    except:
        cats_data['last_exp_da'].append(np.nan)
        
    try:
        cats_data['admin_by'].append(data['results'][0]['drug'][0]['administered_by'])
    except:
        cats_data['admin_by'].append('Unknown')
        
    try:
        cats_data['route'].append(data['results'][0]['drug'][0]['route'])
    except:
        cats_data['route'].append('Unknown')
        
    try:
        cats_data['dosage_form'].append(data['results'][0]['drug'][0]['dosage_form'])
    except:
        cats_data['dosage_form'].append('Unknown')
        
    counter +=1
    #counter is converted to a string and used as a skip value for gathering random records
    str_count = str(counter)
    #base_url = "https://api.fda.gov/animalandveterinary/event.json?search=animal.species=Cat+(animal.breed.breed_component='Domestic Shorthair'+OR+animal.breed.breed_component='Domestic Longhair')&skip=" + str_count
    #base_url = "https://api.fda.gov/animalandveterinary/event.json?search=animal.species=Cat'&skip=" + str_count
    base_url = "https://api.fda.gov/animalandveterinary/event.json?search=animal.breed.breed_component<>'Unknown'&skip=" + str_count

In [5]:
# converting the raw data in the cats dictionary to a dataframe and writing it out to a csv file - mainly because it takes
# a while to run so if the dataframe gets messed up, it can be read from the file instead of running the api again

cat_df = pd.DataFrame.from_dict(cats_data)
cat_df.to_csv("cat_data.csv",index=False)

#print the info of the dataframe

cat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   species          750 non-null    object
 1   gender           750 non-null    object
 2   age              750 non-null    object
 3   weight           750 non-null    object
 4   breed            750 non-null    object
 5   drug             750 non-null    object
 6   outcome          750 non-null    object
 7   date_in          750 non-null    object
 8   react_term_code  750 non-null    object
 9   react_term_name  750 non-null    object
 10  first_exp_da     687 non-null    object
 11  last_exp_da      655 non-null    object
 12  admin_by         750 non-null    object
 13  route            750 non-null    object
 14  dosage_form      750 non-null    object
dtypes: object(15)
memory usage: 88.0+ KB


In [2]:
# read the csv file in 
# changing to just use dogs soon so naming the dataframe appropriately
dog_df = pd.read_csv('cat_data.csv')

#### Begin data analysis and pre-processing

We are just going with dogs this time around so dropping the remaining species.

In [3]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   species          750 non-null    object 
 1   gender           750 non-null    object 
 2   age              750 non-null    float64
 3   weight           750 non-null    float64
 4   breed            750 non-null    object 
 5   drug             750 non-null    object 
 6   outcome          750 non-null    object 
 7   date_in          750 non-null    int64  
 8   react_term_code  750 non-null    int64  
 9   react_term_name  750 non-null    object 
 10  first_exp_da     687 non-null    float64
 11  last_exp_da      655 non-null    float64
 12  admin_by         750 non-null    object 
 13  route            750 non-null    object 
 14  dosage_form      750 non-null    object 
dtypes: float64(4), int64(2), object(9)
memory usage: 88.0+ KB


In [4]:
dog_df.drop(dog_df.index[dog_df['species'] != 'Dog'], inplace = True)

dog_df['species'].value_counts()

Dog    629
Name: species, dtype: int64

In [5]:
dog_df.head(10)

,species,gender,age,weight,breed,drug,outcome,date_in,react_term_code,react_term_name,first_exp_da,last_exp_da,admin_by,route,dosage_form
0,Dog,Female,10.0,5.900,Poodle (unspecified),Spinosad,Recovered/Normal,20120627,334,Vomiting,20120601.0,20120702.0,Animal Owner,Oral,"Tablet, chewable"
1,Dog,Female,7.0,5.900,Terrier - West Highland White,Spinosad,Recovered/Normal,20120627,335,Emesis,20120327.0,20120627.0,Animal Owner,Oral,"Tablet, chewable"
2,Dog,Female,14.0,7.620,Schnauzer (unspecified),Spinosad,Recovered/Normal,20110928,335,Emesis,20110923.0,20110923.0,Animal Owner,Oral,"Tablet, chewable"
3,Dog,Female,0.0,32.660,"['Retriever (unspecified)', 'Dog (other)']",Spinosad,Outcome Unknown,20120601,121,Agitation,20120301.0,20120501.0,Animal Owner,Oral,"Tablet, chewable"
6,Dog,Male,14.0,36.287,Doberman Pinscher,Ivermectin,Outcome Unknown,20150203,305,Digestive tract disorder NOS,20150201.0,20150201.0,Animal Owner,Oral,"Tablet, chewable"
7,Dog,Unknown,0.0,0.000,Dog (unknown),Spinosad,Unknown,20110303,2649,Lack of efficacy (ectoparasite) - flea,NaN,NaN,Unknown,Oral,"Tablet, chewable"
8,Dog,Female,5.0,39.460,Boxer (German Boxer),Imidacloprid,Outcome Unknown,20160420,2082,Emesis (multiple),20160420.0,20160420.0,Other,Other,Solution
9,Dog,Male,10.0,15.000,Crossbred Canine/dog,Recombinant Human Insulin,Outcome Unknown,20160630,119,Restlessness,20150425.0,NaN,Animal Owner,Subcutaneous,Suspension
11,Dog,Male,9.8,5.534,Poodle - Miniature,Ivermectin,Outcome Unknown,20150319,305,Digestive tract disorder NOS,20141201.0,20150301.0,Animal Owner,Oral,"Tablet, chewable"
12,Dog,Male,10.0,33.570,Retriever - Golden,Moxidectin,Outcome Unknown,20150612,130,Crying,20150611.0,20150611.0,Veterinarian,Subcutaneous,Suspension


Start looking at age and weight - to see about dropping those that are 0, since those were unknown's from the api pull

In [6]:
dog_df['age'].value_counts()

0.00     82
2.00     55
3.00     52
4.00     43
7.00     43
8.00     42
5.00     41
6.00     39
10.00    35
11.00    26
9.00     23
1.00     22
12.00    17
13.00    17
14.00    15
15.00    10
16.00     7
1.50      6
17.00     5
22.00     4
4.50      4
18.00     3
21.00     3
2.50      3
1.30      3
6.50      2
1.25      2
3.50      2
2.40      1
8.75      1
3.75      1
12.50     1
9.70      1
12.70     1
9.80      1
2.90      1
19.00     1
4.30      1
4.20      1
23.00     1
11.50     1
10.80     1
4.90      1
7.50      1
5.50      1
20.00     1
1.20      1
10.50     1
16.50     1
6.40      1
9.60      1
Name: age, dtype: int64

Need to drop the dogs with 0 age. Once those are dropped, the age column will be changed to int to combine more of the ages together.

In [7]:
dog_df.drop(dog_df.index[dog_df['age'] == 0], inplace = True)
dog_df['age'].describe()

count    547.000000
mean       7.027971
std        4.516529
min        1.000000
25%        3.000000
50%        6.000000
75%       10.000000
max       23.000000
Name: age, dtype: float64

In [8]:
dog_df[["age"]] = dog_df[["age"]].astype(int)
dog_df['age'].value_counts()

2     60
3     55
4     50
7     44
8     43
5     42
6     42
10    37
1     34
11    27
9     26
12    19
13    17
14    15
15    10
16     8
17     5
22     4
21     3
18     3
20     1
23     1
19     1
Name: age, dtype: int64

Time to evaluate the weight field - dropping the ones with zeroes. Also changing this to int in order to combine weights.

In [9]:
dog_df['weight'].value_counts()

0.000     14
6.800      9
11.340     7
3.629      6
4.990      6
          ..
4.763      1
6.033      1
3.402      1
12.000     1
6.010      1
Name: weight, Length: 343, dtype: int64

In [10]:
dog_df.drop(dog_df.index[dog_df['weight'] == 0], inplace = True)
dog_df['weight'].describe()

count    533.000000
mean      18.705396
std       13.339007
min        0.573000
25%        7.031000
50%       15.876000
75%       28.580000
max       72.575000
Name: weight, dtype: float64

In [11]:
dog_df[["weight"]] = dog_df[["weight"]].astype(int)
dog_df['weight'].value_counts()

9     31
4     30
6     26
3     24
5     23
2     23
7     20
8     18
27    18
30    15
18    13
24    13
32    13
10    13
11    13
15    13
28    12
19    12
17    12
29    12
34    11
31    11
14    11
12    10
26    10
33     9
20     9
13     9
22     9
39     8
35     8
23     8
25     6
1      6
38     6
16     5
40     5
36     5
43     4
45     3
42     3
37     3
46     3
58     3
44     2
52     2
21     2
67     1
48     1
53     1
0      1
41     1
50     1
62     1
72     1
Name: weight, dtype: int64

Changing to int cause a 0 to show again so dropping it

In [12]:
dog_df.drop(dog_df.index[dog_df['weight'] == 0], inplace = True)

Look at the 3 date fields next. First_exp_da and last_exp_da have some nulls. Dropping those to see what is left.

In [13]:
dog_df['last_exp_da'].isnull().sum()

36

In [14]:
dog_df.dropna(inplace=True)

Dropping the nulls leaves 494 rows in the dog_df dataframe

first_exp_da and last_exp_da fields were brought in as float. However, I couldn't convert directly to a date field because the date wasn't fully filled out. I'm going to default to the first day of the month for those missing the day part of the date. If the month is also not fully there, I will default to January, just to get a fully formed date.
- Converting the date fields from float to string first
- Slicing out the individual pieces of the date to create new columns
- Using np.where to update the day and month as mentioned above
- Put the pieces back together
- Convert resulting date to a date field.


In [15]:
dog_df['first_exp_da'] = dog_df['first_exp_da'].astype(str)
dog_df['last_exp_da'] = dog_df['last_exp_da'].astype(str)

In [16]:
dog_df['year_first_exp'] = dog_df.first_exp_da.str.slice(0,4)
dog_df['month_first_exp'] = dog_df.first_exp_da.str.slice(4,6)
dog_df['day_first_exp'] = dog_df.first_exp_da.str.slice(6,8)

dog_df['year_last_exp'] = dog_df.last_exp_da.str.slice(0,4)
dog_df['month_last_exp'] = dog_df.last_exp_da.str.slice(4,6)
dog_df['day_last_exp'] = dog_df.last_exp_da.str.slice(6,8)

In [17]:
dog_df['day_first_exp'] = np.where(dog_df.day_first_exp.lt('01'),'01',dog_df.day_first_exp) 
dog_df['month_first_exp'] = np.where(dog_df.month_first_exp.lt('01'),'01',dog_df.month_first_exp)
dog_df['day_last_exp'] = np.where(dog_df.day_last_exp.lt('01'),'01',dog_df.day_last_exp)
dog_df['month_last_exp'] = np.where(dog_df.month_last_exp.lt('01'),'01',dog_df.month_last_exp)

In [18]:
dog_df['updated_first_exp'] = dog_df['year_first_exp'] + dog_df['month_first_exp'] + dog_df['day_first_exp']
dog_df['updated_last_exp'] = dog_df['year_last_exp'] + dog_df['month_last_exp'] + dog_df['day_last_exp']

In [19]:
dog_df['updated_first_exp'].value_counts()

20130101    7
20120101    7
20140601    4
20130915    4
20130301    4
           ..
20081201    1
20120919    1
20120201    1
20120906    1
20140405    1
Name: updated_first_exp, Length: 406, dtype: int64

In [20]:
dog_df['updated_last_exp'].value_counts()

20170301    4
20170101    4
20180901    4
20140601    3
20170601    3
           ..
20140403    1
20141009    1
20140416    1
20140213    1
20140419    1
Name: updated_last_exp, Length: 410, dtype: int64

In [21]:
dog_df['updated_first_exp'] = pd.to_datetime(dog_df['updated_first_exp'], format="%Y%m%d")
dog_df['updated_last_exp'] = pd.to_datetime(dog_df['updated_last_exp'], format="%Y%m%d")

In [22]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 749
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   species            494 non-null    object        
 1   gender             494 non-null    object        
 2   age                494 non-null    int32         
 3   weight             494 non-null    int32         
 4   breed              494 non-null    object        
 5   drug               494 non-null    object        
 6   outcome            494 non-null    object        
 7   date_in            494 non-null    int64         
 8   react_term_code    494 non-null    int64         
 9   react_term_name    494 non-null    object        
 10  first_exp_da       494 non-null    object        
 11  last_exp_da        494 non-null    object        
 12  admin_by           494 non-null    object        
 13  route              494 non-null    object        
 14  dosage_for

Can now drop the original exp dates and the intermediate ones.

In [23]:
dog_df.drop(['first_exp_da', 'last_exp_da', 'year_first_exp', 'month_first_exp', 'day_first_exp', 'year_last_exp', 'month_last_exp', 'day_last_exp'], axis='columns', inplace=True)

In [24]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 749
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   species            494 non-null    object        
 1   gender             494 non-null    object        
 2   age                494 non-null    int32         
 3   weight             494 non-null    int32         
 4   breed              494 non-null    object        
 5   drug               494 non-null    object        
 6   outcome            494 non-null    object        
 7   date_in            494 non-null    int64         
 8   react_term_code    494 non-null    int64         
 9   react_term_name    494 non-null    object        
 10  admin_by           494 non-null    object        
 11  route              494 non-null    object        
 12  dosage_form        494 non-null    object        
 13  updated_first_exp  494 non-null    datetime64[ns]
 14  updated_la

### DataFrame Manipulation

Continuing clean up of the dataframe

In [25]:
dog_df['outcome'].value_counts()

Outcome Unknown           274
Recovered/Normal          131
Ongoing                    53
Recovered with Sequela     26
Died                        6
Euthanized                  4
Name: outcome, dtype: int64

In [26]:
dog_df['breed'].value_counts()

Retriever - Labrador                                45
Crossbred Canine/dog                                33
Chihuahua                                           18
Beagle                                              15
Shepherd Dog - German                               15
                                                    ..
['Pointer (unspecified)', 'Dog (unknown)']           1
['Terrier (unspecified)', 'Dog (other)']             1
['Shepherd Dog - German', 'Crustacea (unknown)']     1
Terrier - Cairn                                      1
Schipperke                                           1
Name: breed, Length: 152, dtype: int64

There's a lot of strange characters in the breed name. Try to strip them out first before putting the breeds together.

In [27]:
dog_df['breed'] = dog_df['breed'].str.strip('[]')
dog_df['breed'] = dog_df['breed'].str.strip('()')


In [28]:
dog_df['breed'].value_counts()

Retriever - Labrador                              45
Crossbred Canine/dog                              33
Chihuahua                                         18
Beagle                                            15
Shepherd Dog - German                             15
                                                  ..
'Pointer (unspecified)', 'Dog (unknown)'           1
'Terrier (unspecified)', 'Dog (other)'             1
'Shepherd Dog - German', 'Crustacea (unknown)'     1
Terrier - Cairn                                    1
Schipperke                                         1
Name: breed, Length: 152, dtype: int64

Resetting the index after dropping rows

In [29]:
dog_df.reset_index(drop=True, inplace=True)

In [30]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   species            494 non-null    object        
 1   gender             494 non-null    object        
 2   age                494 non-null    int32         
 3   weight             494 non-null    int32         
 4   breed              494 non-null    object        
 5   drug               494 non-null    object        
 6   outcome            494 non-null    object        
 7   date_in            494 non-null    int64         
 8   react_term_code    494 non-null    int64         
 9   react_term_name    494 non-null    object        
 10  admin_by           494 non-null    object        
 11  route              494 non-null    object        
 12  dosage_form        494 non-null    object        
 13  updated_first_exp  494 non-null    datetime64[ns]
 14  updated_la

In [31]:
dog_df['breed'].replace("'Terrier (unspecified)', 'Dog (other)'",'Terrier', inplace=True)
dog_df['breed'].replace("'Retriever - Golden', 'Poodle (unspecified)'", 'Retriever - Golden', inplace=True)
dog_df['breed'].replace("'Spitz - German Pomeranian', 'Chihuahua'", 'Chihuahua', inplace=True)
dog_df['breed'].replace("'Shepherd Dog - German', 'Crustacea (unknown)'",'Shepherd Dog - German', inplace=True)

In [32]:
dog_df['breed'].replace('Terrier - Border', 'Terrier', inplace=True)
dog_df['breed'].replace("'Maltese', 'Japanese Chin (Spaniel)', 'Papillon - Spaniel - Continental Toy (with erect ears or with dropped ears (Phaléne))', 'Shih Tzu'",'Maltese', inplace=True)
dog_df['breed'].replace("'Shepherd Dog - Australian', 'Retriever - Labrador'",'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Chihuahua', 'Crossbred Canine/dog'", 'Chihuahua', inplace=True)

In [33]:
dog_df['breed'].replace("'Spaniel - Cocker American'", 'Spaniel - Cocker American', inplace=True)
dog_df['breed'].replace("'Rottweiler'", 'Rottweiler',inplace=True)
dog_df['breed'].replace("'Poodle - Standard'", 'Poodle - Standard', inplace=True)
dog_df['breed'].replace("Retriever - Labrador', 'Dog (unknown)", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace('Dog (unknown', 'Dog', inplace=True)
dog_df['breed'].replace('Boxer (German Boxer','German Boxer', inplace=True)
dog_df['breed'].replace('Dachshund (unspecified', 'Dachshund', inplace=True)
dog_df['breed'].replace("Retriever - Labrador', 'Poodle (unspecified)", 'Retriever - Labrador Poodle', inplace=True)
dog_df['breed'].replace('Hound (unspecified)', 'Hound', inplace=True)
dog_df['breed'].replace('Hound (unspecified', 'Hound', inplace=True)
dog_df['breed'].replace("'Retriever - Labrador', 'Dog (unknown)', 'Bulldog - French'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Maine Coon', 'Ragdoll'", 'Maine Coon', inplace=True)
dog_df['breed'].replace('Dachshund - Standard Wire-haired', 'Dachshund', inplace=True)
dog_df['breed'].replace("'Retriever - Labrador', 'Shepherd (unspecified)'", 'Retriever - Labrador',inplace=True)
dog_df['breed'].replace('Terrier - Silky', 'Terrier', inplace=True)

In [34]:
dog_df['breed'].replace("'Terrier - Yorkshire', 'Chihuahua'", 'Terrier - Yorkshire',inplace=True)
dog_df['breed'].replace("'Siberian Husky', 'Dog (unknown)'",'Siberian Husky', inplace=True)
dog_df['breed'].replace("'Poodle (unspecified)', 'Dog (unknown)'",'Poodle', inplace=True)
dog_df['breed'].replace("'Mastiff', 'Dog (unknown)'",'Mastiff', inplace=True)
dog_df['breed'].replace("'Shepherd Dog - German', 'Crossbred Canine/dog'", 'Shepherd Dog - German', inplace=True)
dog_df['breed'].replace("'Cattle Dog - Australian (blue heeler, red heeler, Queensland cattledog)', 'Collie - Border'", 'Cattle Dog', inplace=True)
dog_df['breed'].replace("'Cattle Dog - Australian (blue heeler, red heeler, Queensland cattledog)', 'Dog(unknown)'", 'Cattle Dog', inplace=True)
dog_df['breed'].replace("'Cattle Dog - Australian (blue heeler, red heeler, Queensland cattledog)', 'Dog (unknown)'", 'Cattle Dog', inplace=True)
dog_df['breed'].replace("'Maltese','Poodle (unspecified)'",'Maltese',inplace=True)
dog_df['breed'].replace("'Maltese', 'Poodle (unspecified)'",'Maltese', inplace=True)
dog_df['breed'].replace("'Retriever - Labrador', 'Dog (unknown)'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Retriever - Labrador', 'Dog (unknown)'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace('Shepherd (unspecified', 'Shepherd', inplace=True)


In [35]:
dog_df['breed'].replace("'Pug', 'Chihuahua'", 'Pug', inplace=True)
dog_df['breed'].replace("'Spaniel - Cocker American', 'Poodle (unspecified)'", 'Spaniel - Cocker American', inplace=True)
dog_df['breed'].replace("'Shepherd Dog - German', 'Dog (unknown)'", 'Shepherd Dog - German', inplace=True)
dog_df['breed'].replace("'Shepherd (unspecified)', 'Dog (unknown)'", 'Shepherd', inplace=True)
dog_df['breed'].replace("'Alaskan Malamute', 'Retriever - Labrador'", 'Alaskan Malamute', inplace=True)
dog_df['breed'].replace("'Bulldog', 'Pit Bull'", 'Bulldog', inplace=True)
dog_df['breed'].replace("'Chihuahua', 'Dachshund (unspecified)'", 'Chihuahua', inplace=True)
dog_df['breed'].replace("'Chihuahua', 'Greyhound - Italian'", 'Chihuahua', inplace=True)
dog_df['breed'].replace("'Chinese Crested Dog (unspecified)', 'Poodle (unspecified)'", 'Chinese Crested Dog', inplace=True)
dog_df['breed'].replace("'Collie - Border', 'Dog (unknown)'", 'Collie - Border', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Shepherd Dog - German'", 'Shepherd Dog - German', inplace=True)
dog_df['breed'].replace("'Dachshund (unspecified)', 'Dog (unknown)'", 'Dachshund', inplace=True)
dog_df['breed'].replace("'Great Pyrenees', 'Dog (unknown)'", 'Great Pyrenees', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)


In [36]:
dog_df['breed'].replace("'Terrier (unspecified)', 'Dog (unknown)'", 'Terrier', inplace=True)
dog_df['breed'].replace("'Terrier - Yorkshire', 'Dog (unknown)'", 'Terrier - Yorkshire', inplace=True)
dog_df['breed'].replace("'Spitz - German Pomeranian', 'Dog (unknown)'", 'Spitz - German Pomeranian', inplace=True)
dog_df['breed'].replace("'Pit Bull', 'Dog (unknown)'", 'Pit Bull', inplace=True)
dog_df['breed'].replace("'Retriever - Labrador', 'Poodle (unspecified)'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Poodle (unspecified)', 'Terrier (unspecified)'", 'Poodle', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)
dog_df['breed'].replace("'Crossbred Canine/dog', 'Retriever - Labrador'", 'Retriever - Labrador', inplace=True)

In [37]:
dog_df['breed'].value_counts()

Retriever - Labrador             64
Crossbred Canine/dog             33
Shepherd Dog - German            21
Chihuahua                        21
Terrier - Yorkshire              16
                                 ..
'Rottweiler', 'Dog (unknown)'     1
Shar Pei                          1
'Pug', 'Shih Tzu'                 1
Griffon - Brussels                1
Schipperke                        1
Name: breed, Length: 122, dtype: int64

In [38]:
# Get the count of each value
value_counts = dog_df['breed'].value_counts()

# Select the values where the count is less than 2
to_remove = value_counts[value_counts < 2].index

# Keep rows where the breed column is not in to_remove
dog_df = dog_df[~dog_df.breed.isin(to_remove)]

In [39]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 1 to 493
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   species            434 non-null    object        
 1   gender             434 non-null    object        
 2   age                434 non-null    int32         
 3   weight             434 non-null    int32         
 4   breed              434 non-null    object        
 5   drug               434 non-null    object        
 6   outcome            434 non-null    object        
 7   date_in            434 non-null    int64         
 8   react_term_code    434 non-null    int64         
 9   react_term_name    434 non-null    object        
 10  admin_by           434 non-null    object        
 11  route              434 non-null    object        
 12  dosage_form        434 non-null    object        
 13  updated_first_exp  434 non-null    datetime64[ns]
 14  updated_la

In [40]:
dog_df['react_term_code'].value_counts()

334     70
335     40
305     34
1039    28
2430    24
        ..
193      1
998      1
124      1
2071     1
837      1
Name: react_term_code, Length: 103, dtype: int64

In [41]:
dog_df['react_term_name'].value_counts()

Vomiting                                       70
Emesis                                         40
Digestive tract disorder NOS                   34
Lack of efficacy - NOS                         28
Lack of efficacy (endoparasite) - heartworm    23
                                               ..
Decreased appetite                              1
Urinary tract infection                         1
General illness                                 1
Skin irritation                                 1
Abnormal breathing                              1
Name: react_term_name, Length: 106, dtype: int64

In [42]:
dog_df['react_term_name'].replace('Emesis (multiple)', 'Emesis', inplace=True)

In [43]:
dog_df['drug'].value_counts()

Spinosad                                                         151
Ivermectin                                                        72
Milbemycin Oxime                                                  38
Afoxolaner                                                        22
Moxidectin                                                        13
Nitenpyram                                                         9
Imidacloprid                                                       8
Melarsomine Dihydrochloride Injection                              7
Milbemycin Oxime, Lufenuron                                        7
Sarolaner                                                          7
Oclacitinib Maleate                                                6
Ivermectin 272Mcg, Pyrantel Pamoate 228Mg, Praziquantel 228Mg      6
Fluralaner Chew Tablets                                            6
Ivermectin/Pyrantel Pamoate Chewable 136Mcg/326Mg                  5
Ivermectin/Pyrantel Pamoate Chewab

Decided to drop rows that only have count of 1 drug

In [44]:
# Get the count of each value
value_counts2 = dog_df['drug'].value_counts()

# Select the values where the count is less than 2
to_remove2 = value_counts2[value_counts2 < 2].index

# Keep rows where the breed column is not in to_remove
dog_df = dog_df[~dog_df.drug.isin(to_remove2)]

In [45]:
dog_df['drug'].value_counts()

Spinosad                                                         151
Ivermectin                                                        72
Milbemycin Oxime                                                  38
Afoxolaner                                                        22
Moxidectin                                                        13
Nitenpyram                                                         9
Imidacloprid                                                       8
Sarolaner                                                          7
Milbemycin Oxime, Lufenuron                                        7
Melarsomine Dihydrochloride Injection                              7
Fluralaner Chew Tablets                                            6
Oclacitinib Maleate                                                6
Ivermectin 272Mcg, Pyrantel Pamoate 228Mg, Praziquantel 228Mg      6
Ivermectin/Pyrantel Pamoate Chewable 136Mcg/326Mg                  5
Pyrantel Pamoate;Sarolaner        

Finally, we look at the "Gender" category:

In [46]:
dog_df['gender'].value_counts()

Female     232
Male       178
Unknown      3
Mixed        2
Name: gender, dtype: int64

There are four categories, but we didn't drop "Unknown" and "Mixed" from the set. "Unknown" comes from lack of information, and "Mixed" may or may not be a hermaphrodite. Since there are only 5 total, probably could have dropped them but thought it was a point of interest.


In [53]:
dog_df.to_csv("dog_data.csv",index=False)

After all of the preprocessing was completed, the dog_df dataframe was written to a csv file. Now reading that back in to use for database work.

In [54]:
dog_df = pd.read_csv('dog_data.csv')

In order to have ID's for each of the resulting tables, I am going to add columns to the original dataframe first - need one for drug_id (needs to use columns planning to be put into that table), breed, gender+age+weight, and reaction (outcome+react_term_name+updated_first_exp+updated_last_exp)

In [55]:
from sklearn.preprocessing import LabelEncoder

class_labels = LabelEncoder()
dog_df['breed_id']=class_labels.fit_transform(dog_df['breed'].values)


In [56]:
#dog_df['drug_info']=dog_df.dog_df[['drug','admin_by','route','dosage_form']]
dog_df['drug_info'] = dog_df['drug'] + dog_df['admin_by'] + dog_df['route'] + dog_df['dosage_form']

In [57]:
dog_df['drug_info'].value_counts()

SpinosadAnimal OwnerOralTablet, chewable                                                         150
IvermectinAnimal OwnerOralTablet, chewable                                                        49
Milbemycin OximeAnimal OwnerOralTablet, chewable                                                  25
IvermectinUnknownOralTablet, chewable                                                             20
AfoxolanerAnimal OwnerOralTablet, chewable                                                        16
Milbemycin OximeAnimal OwnerOralTablet                                                            13
NitenpyramAnimal OwnerOralTablet                                                                   8
ImidaclopridAnimal OwnerTopicalSolution                                                            7
Milbemycin Oxime, LufenuronAnimal OwnerOralTablet                                                  7
Oclacitinib MaleateAnimal OwnerOralTablet                                                  

In [58]:
dog_df['drug_id']=class_labels.fit_transform(dog_df['drug_info'].values)

In [59]:
dog_df['breed_specs']=dog_df['gender'] + dog_df['age'].astype(str) + dog_df['weight'].astype(str) + dog_df['breed_id'].astype(str)

In [60]:
dog_df['breed_specs'].value_counts()

Male73939      2
Male52013      2
Female51313    2
Female2427     2
Female4227     2
              ..
Female11248    1
Female11155    1
Male5652       1
Female12741    1
Female42913    1
Name: breed_specs, Length: 408, dtype: int64

In [61]:
dog_df['breed_specs_id']=class_labels.fit_transform(dog_df['breed_specs'].values)


In [62]:
dog_df['react_info']=dog_df['outcome']+dog_df['react_term_name']+dog_df['updated_first_exp'].astype(str)+dog_df['updated_last_exp'].astype(str)
dog_df['react_id']=class_labels.fit_transform(dog_df['react_info'].values)

In [63]:
dog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   species            415 non-null    object
 1   gender             415 non-null    object
 2   age                415 non-null    int64 
 3   weight             415 non-null    int64 
 4   breed              415 non-null    object
 5   drug               415 non-null    object
 6   outcome            415 non-null    object
 7   date_in            415 non-null    int64 
 8   react_term_code    415 non-null    int64 
 9   react_term_name    415 non-null    object
 10  admin_by           415 non-null    object
 11  route              415 non-null    object
 12  dosage_form        415 non-null    object
 13  updated_first_exp  415 non-null    object
 14  updated_last_exp   415 non-null    object
 15  breed_id           415 non-null    int32 
 16  drug_info          415 non-null    object
 1

Now that I've added the id's I need, I am removing the intermediate columns created during this process (breed_specs, react_info, and drug_info)

In [64]:
dog_df.drop(['breed_specs', 'react_info', 'drug_info'], axis='columns', inplace=True)


Decided to split the dog_df into multiple dataframes to make loading to postgres easier.
- Breed will contain breed_id and breed
- Breed_specs will contain breed_specs_id, breed_id, gender, age, weight
- Drug_t will contain drug_id, drug, admin_by, route, dosage_form
- React will contain react_id, react_term_name, outcome, updated_first_exp, updated_last_exp

In [65]:
breed_df = dog_df.filter(['breed_id','breed'], axis=1)
breed_specs_df = dog_df.filter(['breed_id', 'gender', 'age', 'weight'], axis=1)
drug_t_df = dog_df.filter(['drug_id', 'drug', 'admin_by', 'route', 'dosage_form'], axis=1)
react_df = dog_df.filter(['react_id', 'react_term_name', 'outcome', 'updated_first_exp', 'updated_last_exp'], axis=1)

In [66]:
breed_specs_df.insert(0, 'breed_specs_id', range(0, 0 + len(breed_specs_df)))

In [67]:
breed_drug_df = dog_df.filter(['breed_id', 'drug_id'], axis=1)
breed_drug_df.insert(0, 'b_drug_id', range(0, 0 + len(breed_drug_df)))

drug_react_df = dog_df.filter(['react_id', 'drug_id'], axis=1)
drug_react_df.insert(0, 'd_react_id', range(0, 0 + len(drug_react_df)))



In [68]:
breed_specs_df.head()

,breed_specs_id,breed_id,gender,age,weight
0,0,58,Female,7,5
1,1,41,Female,14,7
2,2,18,Male,14,36
3,3,21,Female,5,39
4,4,34,Male,9,5


In [69]:
breed_drug_df.head()

,b_drug_id,breed_id,drug_id
0,0,58,54
1,1,41,54
2,2,18,22
3,3,21,16
4,4,34,22


In [70]:
drug_react_df.head()

,d_react_id,react_id,drug_id
0,0,324,54
1,1,318,54
2,2,97,22
3,3,109,16
4,4,94,22


need to drop duplicates from some of these dataframes since they are "domain tables"

In [71]:
breed_df.value_counts()

breed_id  breed                
39        Retriever - Labrador     62
13        Crossbred Canine/dog     30
47        Shepherd Dog - German    21
8         Chihuahua                20
2         Beagle                   15
                                   ..
28        Mastiff                   2
30        Mountain Cur              2
33        Poodle                    2
61        Weimaraner                2
20        Dog (other                1
Length: 62, dtype: int64

In [72]:

breed_df=breed_df.drop_duplicates()


In [73]:
breed_df.value_counts()

breed_id  breed                          
0         Akita                              1
46        Shepherd Dog - Belgian Malinois    1
33        Poodle                             1
34        Poodle - Miniature                 1
35        Poodle - Standard                  1
                                            ..
25        Hound                              1
26        Hound - Basset                     1
27        Maltese                            1
28        Mastiff                            1
61        Weimaraner                         1
Length: 62, dtype: int64

In [74]:
drug_t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   drug_id      415 non-null    int32 
 1   drug         415 non-null    object
 2   admin_by     415 non-null    object
 3   route        415 non-null    object
 4   dosage_form  415 non-null    object
dtypes: int32(1), object(4)
memory usage: 14.7+ KB


In [75]:
breed_drug_df.value_counts()

b_drug_id  breed_id  drug_id
0          58        54         1
273        13        22         1
283        10        19         1
282        13        15         1
281        51        39         1
                               ..
135        48        54         1
134        53        54         1
133        47        54         1
132        48        54         1
414        13        54         1
Length: 415, dtype: int64

In [76]:
drug_react_df.value_counts()

d_react_id  react_id  drug_id
0           324       54         1
273         178       22         1
283         58        19         1
282         250       15         1
281         308       39         1
                                ..
135         251       54         1
134         232       54         1
133         107       54         1
132         304       54         1
414         369       54         1
Length: 415, dtype: int64

In [77]:
breed_specs_df.value_counts()

breed_specs_id  breed_id  gender  age  weight
0               58        Female  7    5         1
273             13        Female  2    15        1
283             10        Male    2    22        1
282             13        Male    13   20        1
281             51        Female  15   7         1
                                                ..
135             48        Female  2    3         1
134             53        Male    1    5         1
133             47        Male    5    24        1
132             48        Female  4    8         1
414             13        Female  4    29        1
Length: 415, dtype: int64

In [78]:
drug_t_df=drug_t_df.drop_duplicates() 
breed_drug_df=breed_drug_df.drop_duplicates()
drug_react_df=drug_react_df.drop_duplicates()
breed_specs_df=breed_specs_df.drop_duplicates()

In [79]:
drug_t_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 400
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   drug_id      57 non-null     int32 
 1   drug         57 non-null     object
 2   admin_by     57 non-null     object
 3   route        57 non-null     object
 4   dosage_form  57 non-null     object
dtypes: int32(1), object(4)
memory usage: 2.4+ KB


In [80]:
drug_t_df.head()

,drug_id,drug,admin_by,route,dosage_form
0,54,Spinosad,Animal Owner,Oral,"Tablet, chewable"
2,22,Ivermectin,Animal Owner,Oral,"Tablet, chewable"
3,16,Imidacloprid,Other,Other,Solution
5,40,Moxidectin,Veterinarian,Subcutaneous,Suspension
7,42,Nitenpyram,Animal Owner,Oral,Tablet


Since there are commas in the actual drug name field as well as the dosage, replacing the commas with periods to not mess the csv file up later. Replacing commas in breed and react_term_name with spaces.

In [81]:
drug_t_df['drug']=drug_t_df['drug'].str.replace(',','.')
drug_t_df['dosage_form']=drug_t_df['dosage_form'].str.replace(',','.')
breed_df['breed']=breed_df['breed'].str.replace(',',' ')
react_df['react_term_name']=react_df['react_term_name'].str.replace(',',' ')

In [82]:
breed_drug_df.value_counts()

b_drug_id  breed_id  drug_id
0          58        54         1
273        13        22         1
283        10        19         1
282        13        15         1
281        51        39         1
                               ..
135        48        54         1
134        53        54         1
133        47        54         1
132        48        54         1
414        13        54         1
Length: 415, dtype: int64

In [83]:
drug_react_df.value_counts()

d_react_id  react_id  drug_id
0           324       54         1
273         178       22         1
283         58        19         1
282         250       15         1
281         308       39         1
                                ..
135         251       54         1
134         232       54         1
133         107       54         1
132         304       54         1
414         369       54         1
Length: 415, dtype: int64

In [84]:
breed_specs_df.value_counts()

breed_specs_id  breed_id  gender  age  weight
0               58        Female  7    5         1
273             13        Female  2    15        1
283             10        Male    2    22        1
282             13        Male    13   20        1
281             51        Female  15   7         1
                                                ..
135             48        Female  2    3         1
134             53        Male    1    5         1
133             47        Male    5    24        1
132             48        Female  4    8         1
414             13        Female  4    29        1
Length: 415, dtype: int64

Write each of the new dataframes to csv files.

In [85]:
breed_df.to_csv("breed.csv",index=False)


In [86]:
breed_specs_df.to_csv("breed_specs.csv",index=False)
drug_t_df.to_csv("drug_t.csv",index=False)
react_df.to_csv("react.csv",index=False)
breed_drug_df.to_csv("breed_drug.csv",index=False)
drug_react_df.to_csv("drug_react.csv",index=False)

## Database model 


Begin the database work - create the tables, then populate with data. 

In [88]:
#Import the python driver for PostgreSQL
import psycopg2
from passwords import password


### Create tables

In [98]:
def create_tables():
    """ Use create commands to add tables to dogs database in postgres"""
    
    commands = (
        """
        CREATE TABLE Drug_t(
        drug_id SERIAL PRIMARY KEY,
        drug CHAR(65),
        admin_by CHAR(30),
        route CHAR(20),
        dosage_form CHAR(20))
        """,
        """
        CREATE TABLE Breed_type(
        breed_id SERIAL PRIMARY KEY,
        breed CHAR(75))
        """,
        """
        CREATE TABLE Breed_specs(
        breed_specs_id SERIAL PRIMARY KEY,
        breed_id SERIAL REFERENCES Breed_type(breed_id),
        gender CHAR(10),
        age INTEGER,
        weight INTEGER)
        """,
        """
        CREATE TABLE React(
        react_id SERIAL PRIMARY KEY,
        react_term_name CHAR(75), 
        outcome CHAR(30), 
        updated_first_exp DATE,
        updated_last_exp DATE)
        """,
        """
        CREATE TABLE Breed_drug(
        b_drug_id SERIAL PRIMARY KEY,
        breed_id SERIAL REFERENCES Breed_type(breed_id), 
        drug_id SERIAL REFERENCES Drug_t(drug_id)) 
        """,
        """
        CREATE TABLE Drug_react(
        d_react_id SERIAL PRIMARY KEY,
        react_id SERIAL REFERENCES React(react_id),
        drug_id SERIAL REFERENCES Drug_t(drug_id))
        """
        )
    
    
    #Create a connection credentials to the PostgreSQL database
    try:
        conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
        print("Connection established")
        
        #Create a cursor connection object to a PostgreSQL instance 
        cur = conn.cursor()
        print("Cursor opened")
    
        for command in commands:
            cur.execute(command)
    
        print("Tables created")
    
        #Commit transaction and prints the result successfully
        conn.commit()
        print ("Commit successful")

    #Handle the error throws by the command that is useful when using python while working with PostgreSQL
    except(Exception, psycopg2.Error) as error:
        print("Error connecting to PostgreSQL database", error)
        conn = None

    #Close the database connection
    finally:
        if(conn != None):
            cur.close()
            conn.close()
            print("PostgreSQL cursor & connection is now closed")
            
if __name__ == '__main__':
    create_tables()

Connection established
Cursor opened
Tables created
Commit successful
PostgreSQL cursor & connection is now closed


## Tables have been created, ready to insert data

Open the csv files and use copy_from to load the data. Executing each table in a separate cell, in case of errors.

In [99]:
def load_table_func(conn,csv_file,table_na):
    cur = conn.cursor()
    print("Cursor opened for processing csv ",csv_file)
    
    with open(csv_file,'r') as i:
        # Skip the header row
        next(i)
        
        # copy the table
        cur.copy_from(i,table_na,sep=',',null='')
    conn.commit()
    cur.close()
    conn.close()

In [100]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'breed.csv','breed_type')


Connection established
Cursor opened for processing csv  breed.csv


Drug_t next

In [101]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'drug_t.csv','drug_t')

Connection established
Cursor opened for processing csv  drug_t.csv


Breed_specs

In [102]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'breed_specs.csv','breed_specs')

Connection established
Cursor opened for processing csv  breed_specs.csv


React

In [103]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'react.csv','react')

Connection established
Cursor opened for processing csv  react.csv


breed_drug

In [104]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'breed_drug.csv','breed_drug')

Connection established
Cursor opened for processing csv  breed_drug.csv


drug_react

In [105]:
conn = psycopg2.connect(user = "postgres", password = password, database = "dogs")
print("Connection established")

load_table_func(conn,'drug_react.csv','drug_react')

Connection established
Cursor opened for processing csv  drug_react.csv


Data is loaded in tables - I am sure I could have consolidated that a little more. Now need to do some selects to show and then some joins to show my data.

In [107]:
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    first_qry = """SELECT DISTINCT(drug) 
                FROM drug_t"""
    
    first_res = pd.read_sql_query(first_qry,conn)

first_res.head()

,drug
0,Melarsomine Dihydrochloride ...
1,Pyrantel Pamoate;Sarolaner ...
2,Nitenpyram ...
3,Maropitant Citrate ...
4,Ivermectin/Pyrantel Pamoate Chewable 272Mcg/65...


In [114]:
# how many times do which drugs produce the reaction of vomiting?
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    sec_qry = """SELECT COUNT(react_term_name) as react_count, d.drug 
            FROM react as r
            INNER JOIN drug_react as dr
            ON r.react_id = dr.react_id
            INNER JOIN drug_t as d
            ON dr.drug_id = d.drug_id
            WHERE react_term_name = 'Vomiting'
            GROUP BY d.drug
            ORDER BY react_count DESC"""
    sec_res = pd.read_sql_query(sec_qry,conn)

sec_res.head()

,react_count,drug
0,32,Spinosad ...
1,7,Afoxolaner ...
2,4,Milbemycin Oxime ...
3,3,Pyrantel Pamoate ...
4,2,Moxidectin ...


In [111]:
# what are the different breeds that can be found in the dogs database?
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    third_qry = """SELECT breed FROM breed_type"""
    third_res = pd.read_sql_query(third_qry,conn)

third_res

,breed
0,Terrier - West Highland White ...
1,Schnauzer (unspecified ...
2,Doberman Pinscher ...
3,German Boxer ...
4,Poodle - Miniature ...
...,...
57,Poodle ...
58,American Pit Bull Terrier ...
59,Coonhound - Black and Tan ...
60,Great Pyrenees ...


In [112]:
# which breed shows most often in the dogs database?
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    fourth_qry = """SELECT COUNT(breed) AS breed_count, breed 
            FROM breed_type AS bt
            INNER JOIN breed_specs AS bs
            ON bt.breed_id = bs.breed_id
            GROUP BY breed
            ORDER BY breed_count DESC"""
    fourth_res = pd.read_sql_query(fourth_qry,conn)

fourth_res

,breed_count,breed
0,62,Retriever - Labrador ...
1,30,Crossbred Canine/dog ...
2,21,Shepherd Dog - German ...
3,20,Chihuahua ...
4,15,Beagle ...
...,...,...
57,2,Dachshund - Miniature ...
58,2,Terrier (unspecified ...
59,2,Mountain Cur ...
60,2,Collie (unspecified ...


In [116]:
# which drugs are given to the different breeds most often?
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    fifth_qry = """SELECT COUNT(d.drug) AS drug_count, d.drug, breed
            FROM drug_t AS d
            INNER JOIN breed_drug AS bd
            ON d.drug_id = bd.drug_id
            INNER JOIN breed_type AS bt
            ON bd.breed_id = bt.breed_id
            GROUP BY d.drug, breed
            ORDER BY drug_count DESC"""
    fifth_res = pd.read_sql_query(fifth_qry,conn)
    
fifth_res.head()

,drug_count,drug,breed
0,19,Spinosad ...,Retriever - Labrador ...
1,13,Ivermectin ...,Retriever - Labrador ...
2,10,Spinosad ...,Crossbred Canine/dog ...
3,10,Milbemycin Oxime ...,Retriever - Labrador ...
4,9,Spinosad ...,Terrier - Yorkshire ...


In [118]:
# who administers the drug to each dog?
with psycopg2.connect(user = "postgres", password = password, database = "dogs") as conn:
    sixth_qry = """SELECT DISTINCT(admin_by), COUNT(admin_by) 
            FROM drug_t AS d
            GROUP BY DISTINCT(admin_by)"""
    sixth_res = pd.read_sql_query(sixth_qry,conn)
    
sixth_res

,admin_by,count
0,Veterinarian,10
1,Other Health Care Professional,1
2,Patient,2
3,Unknown,10
4,Other,5
5,Animal Owner,29
